In [1]:
from unityagents import UnityEnvironment
from collections import deque
from maddpgtennis import DDPG, ReplayBuffer, MADDPG

import matplotlib.pyplot as plt
import numpy as np
import torch
import time as t_


Bad key "text.kerning_factor" on line 4 in
C:\Users\Prabowo Setiawan\miniconda3\envs\drlnd\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
env = UnityEnvironment(file_name='Tennis_Windows_x86_64/Tennis.exe', no_graphics=False)

brain_name = env.brain_names[0]
brain = env.brains[brain_name]

env_info = env.reset(train_mode=True)[brain_name]
n_agents = len(env_info.agents)
print('Number of agents:', n_agents)
n_action = brain.vector_action_space_size
print('Size of each action:', n_action)
states = env_info.vector_observations
n_state = states.shape[1]
print('There are {} agents. Each observes a state with length: {}'.format(states.shape[0], n_state))

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: TennisBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 8
        Number of stacked Vector Observation: 3
        Vector Action space type: continuous
        Vector Action space size (per agent): 2
        Vector Action descriptions: , 


Number of agents: 2
Size of each action: 2
There are 2 agents. Each observes a state with length: 24


In [3]:
ma_ddpg = MADDPG(n_state=n_state, n_action=n_action, n_agents=n_agents)

In [4]:
ma_ddpg.agents_[0].act_local.load_state_dict(torch.load('saved_model/agent0_DDPG_act_tennis.pth'))
ma_ddpg.agents_[0].crit_local.load_state_dict(torch.load('saved_model/agent0_DDPG_crit_tennis.pth'))
ma_ddpg.agents_[1].act_local.load_state_dict(torch.load('saved_model/agent1_DDPG_act_tennis.pth'))
ma_ddpg.agents_[1].crit_local.load_state_dict(torch.load('saved_model/agent1_DDPG_crit_tennis.pth'))

In [5]:
for i in range(10):
    env_info = env.reset(train_mode=False)[brain_name]
    state = env_info.vector_observations
    while True:
        actions = []
        sel_actions = ma_ddpg.action_(states)
        act_ = torch.stack(sel_actions).cpu().detach()
        actions = np.clip(act_.numpy(), -1, 1)
        env_info = env.step(actions)[brain_name]
        states = env_info.vector_observations
        dones = env_info.local_done
        if np.any(dones):
            break

In [6]:
env.close()